# Задание

1. Написать приложение, которое собирает основные новости с сайтов `mail.ru, lenta.ru, yandex.news`
2. Сложить все новости в БД

Для парсинга использовать xpath. Структура данных должна содержать:
- название источника,
- наименование новости,
- ссылку на новость,
- дата публикации

### Подключение библиотек

In [169]:
import requests
from pprint import pprint
from lxml import html
from pymongo import MongoClient
from datetime import datetime
import json

---

### Запуск сервера

In [170]:
try:
    client = MongoClient('localhost',27017)
    print('connected successfully')
except:
    print('bad connection')

db = client['database_news_dz4']  # создание БД
news_from_3_sites = db.news_from_3_sites  # содание коллекции

connected successfully


---

In [171]:
main_link_mail = 'https://news.mail.ru/go-mobile/'
main_link_ya = 'https://yandex.ru/news/'
main_link_lenta = 'https://lenta.ru'

header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'}

def tree(main_link):
    response = requests.get(main_link, headers=header).text
    tree = html.fromstring(response)
    return tree

In [172]:
# - название источника,
# - наименование новости,
# - ссылку на новость,
# - дата публикации

In [173]:
# news_all = []

### Сбор новостей

#### https://yandex.ru/news/

In [174]:
news_list_ya = tree(main_link_ya).xpath("//div[@class='story__topic']")

In [175]:
for news in news_list_ya:
    news_data={}
    
    name = str(news.xpath(".//h2[@class='story__title']/a/text()")[0])
    origin = str(news.xpath("..//div[@class='story__info']/div[1]/text()")[0])[:-5]
    href = 'https://yandex.ru' + str(news.xpath(".//h2[@class='story__title']/a/@href")[0])
    date_time = str(datetime.now())[:10] + str(news.xpath("..//div[@class='story__info']/div[1]/text()")[0])[-5:]    
    
    news_data['1 название источника'] = origin
    news_data['2 наименование новости'] = name
    news_data['3 ссылка на новость'] = href
    news_data['4 дата публикации'] = date_time
    
    news_from_3_sites.insert_one(news_data)

#### https://lenta.ru/

In [176]:
news_list_lenta = tree(main_link_lenta).xpath("//section[@class='row b-top7-for-main js-top-seven']/div[@class='span4']/div")

In [177]:
for news in news_list_lenta[:-1]:
    news_data={}
    
    name = str(news.xpath(".//a[1]/text()")[0])
    origin = 'lenta.ru'
    href = main_link_lenta + str(news.xpath("./a/@href")[0])
    date_time = str(news.xpath(".//a[1]/time[1]/@title")[0])
      
    news_data['1 название источника'] = origin
    news_data['2 наименование новости'] = name
    news_data['3 ссылка на новость'] = href
    news_data['4 дата публикации'] = date_time
    
    news_from_3_sites.insert_one(news_data)

#### https://news.mail.ru/

In [178]:
tree_mail = tree(main_link_mail)

In [179]:
news_mail_photo = tree_mail.xpath("//a[@class='photo']")
news_list_mail = tree_mail.xpath("//a[@class='item item_side_left entity']")

In [180]:
def photo_mail():
    news_data={}
    
    name = news_mail_photo[0].xpath("//span[@class='photo__title']/text()")[0]
    href = 'https://news.mail.ru' + news_mail_photo[0].xpath("./@href")[0]
    origin = tree(href).xpath("//a[@class='link color_gray breadcrumbs__link']//span[@class='link__text']/text()")[0]
    date_time = tree(href).xpath("//span[@class='note__text breadcrumbs__text js-ago']/@datetime")[0]
    
    news_data['1 название источника'] = origin
    news_data['2 наименование новости'] = name
    news_data['3 ссылка на новость'] = href
    news_data['4 дата публикации'] = date_time
    
    news_from_3_sites.insert_one(news_data)

photo_mail()

In [185]:
def list_mail():
    for news_one in news_list_mail[:10]:
        news_data={}
        href = 'https://news.mail.ru' + news_one.xpath("./@href")[0]
        name = tree(href).xpath("//h1[@class='hdr__inner']/text()")[0]
        origin = tree(href).xpath("//a[@class='link color_gray breadcrumbs__link']//span[@class='link__text']/text()")[0]
        date_time = tree(href).xpath("//span[@class='note__text breadcrumbs__text js-ago']/@datetime")[0]
    
        news_data['1 название источника'] = origin
        news_data['2 наименование новости'] = name
        news_data['3 ссылка на новость'] = href
        news_data['4 дата публикации'] = date_time
        
#         pprint(news_data)
        news_from_3_sites.insert_one(news_data)

list_mail()

### Проверка

In [186]:
from pandas import DataFrame

cursor = news_from_3_sites.find({})
DataFrame(list(cursor)).drop(['_id'], axis=1)

,1 название источника,2 наименование новости,3 ссылка на новость,4 дата публикации
0,RT на русском,Силуанов разъяснил налог на проценты по банков...,https://yandex.ru/news/story/Siluanov_razyasni...,2020-03-2702:01
1,Ведомости,Власти Москвы могут продлить выходные из-за ко...,https://yandex.ru/news/story/Vlasti_Moskvy_mog...,2020-03-2702:06
2,Lenta.ru,В России ответили на критику помощи Италии в б...,https://yandex.ru/news/story/V_Rossii_otvetili...,2020-03-2701:40
3,Газета.Ru,Размер штрафа за нарушение карантина может дос...,https://yandex.ru/news/story/Razmer_shtrafa_za...,2020-03-2701:39
4,RT на русском,США назначили награду в $15 млн за помощь в ар...,https://yandex.ru/news/story/SSHA_naznachili_n...,2020-03-2702:14
...,...,...,...,...
81,РИА Новости,Украина с пятницы закроет границы,https://news.mail.ru/politics/41113911/,2020-03-26T20:24:51+03:00
82,BBC News Русская служба,«Необычно высокая активность». Британский флот...,https://news.mail.ru/politics/41110455/,2020-03-26T19:36:58+03:00
83,РИА Новости,США обвинили Мадуро в наркоторговле и объявили...,https://news.mail.ru/politics/41112457/,2020-03-26T18:41:43+03:00
84,Lenta.Ru,Раскрыты детали налога на вклады в России,https://news.mail.ru/economics/41111979/,2020-03-26T18:20:01+03:00


---

При необходимости - очищение коллекции

In [ ]:
# news_from_3_sites.delete_many({})